<footer id="footer"></footer>

<h1 align='center'>Embeddings</h1>

<p align="center"><img src="https://drive.google.com/uc?id=1o8AW-j6dF5btMJvRHyA05U5zqW-IZm-U" alt="w2v" border="0"></a></p>







<b> **Baseline** подготовлен [Deep learning school](#https://www.dlschool.org/pro-track)

---

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


# Embeddings

Мы с помощью эмбеддингов решим задачу семантической классификации твитов. Для этого мы воспользуемся предобученными эмбеддингами `word2vec`.



Для начала скачаем датасет для семантической классификации твитов:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/data/test/archive.zip

Archive:  /content/drive/MyDrive/data/test/archive.zip
  inflating: training.1600000.processed.noemoticon.csv  


Импортируем нужные библиотеки:

In [ ]:
import re
import math
import random
import string
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import torch
import nltk
import gensim
import torch.optim as optim
import gensim.downloader as api
from torch.utils.data import Dataset, random_split
import torch.nn as nn
from torch.optim import Adam
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader
from sklearn.manifold import TSNE
import warnings
from torch.nn import functional as F
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
tqdm.pandas()
warnings.filterwarnings('ignore')

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
device

'cuda'

In [ ]:
data = (pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin",
                   header=None, names=["emotion", "id", "date", "flag", "user", "text"]))

Посмотрим на данные:

In [ ]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [ ]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как видим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [ ]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [ ]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][1].lower())
print(" ".join(line))

@ jhicks i will think of you on sunday ! who are you by the way ? i am val .


In [ ]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

jhicks will think sunday


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [ ]:
word2vec = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.data = data
        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(self.word2vec.vectors, axis=0)
        self.std = np.std(self.word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])
        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
       line = self.tokenizer.tokenize(text.lower())
       tokens = [w for w in line if all(c not in \
                                               string.punctuation for c in w) and len(w) > 3]
       return tokens

    def get_embeddings_(self, tokens):
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std for w \
                      in tokens if w in self.word2vec and len(w) > 3]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


## Average embedding
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии.

In [ ]:
print(dev.__len__())

1280000


In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


Сделаем визуализацию полученных векторов твитов тренировочного `(dev)` датасета. Так мы увидим, насколько хорошо твиты с разными `target` значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

In [ ]:
tsne = TSNE(n_components=2, init='random',
            learning_rate='auto')
examples["transformed_features"] = tsne.fit_transform(np.asarray(examples['features']))

In [ ]:
def draw_vectors(features, label, opacity=.72):
  fig = px.scatter(features, label,
           color=label, opacity=opacity,
                 labels={
                     'color': "Класс "
                 },
                title="Отображение распределений признаков по классам с t-SNE")\
                .for_each_trace(lambda t: t.update(name=t.name.replace("=",": "))
                )   
  fig.update_traces(marker=dict(size=10,
                                line=dict(width=1,
                                          color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
  fig.update_yaxes(visible=False, showticklabels=False)
  fig.update_xaxes(visible=False, showticklabels=False)
  fig.show();

In [ ]:
draw_vectors(examples["transformed_features"], examples["targets"])

На визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных


In [ ]:
def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}

In [ ]:
batch_size = 1024
num_workers = 2

train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [ ]:
def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    history = []
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device).unsqueeze(1).float()
        optimizer.zero_grad()
        logits = model(features)
        loss = criterion(logits, targets)
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    
def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_acc = 0
    running_loss = 0
    processed_size = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device).unsqueeze(1).float()
            logits = model(features)
            loss = criterion(logits, targets)
            probabilities = torch.sigmoid(logits)
            preds = ((probabilities > 0.5) * 1.)
            log_acc = (preds == targets.data).sum().item() / targets.size(0)
            mean_acc += (preds == targets.data).sum().item() 
            processed_size += targets.size(0)
            running_loss += loss.item() * targets.size(0)
            pbar.set_description(f"Test Loss: {loss.item():.4}, Test Acc: {log_acc:.2%}")
    mean_acc = round((mean_acc / processed_size) * 100, 2)
    mean_loss = running_loss / processed_size
    pbar.set_description(f"Test Loss: {mean_loss:.4}, Test Acc: {mean_acc:.2%}")
    return {"Test Loss": mean_loss, "Test Acc, %": mean_acc}

Создадим модель, оптимизатор и целевую функцию. Сделаем сеть с 5 слоями, добавим нормализацию и `drop_out`. Данную архитектуру будем применять для всех экспериментов с целью понимания степени улчшений в случае применения того или иного подхода


In [ ]:
class TextClassficator(nn.Module):
    def __init__(self, x):
      super().__init__()

      self.linear_01 = self.linear_(x, 200)
      self.linear_02 = self.linear_(200, 100)
      self.linear_03 = self.linear_(100, 100)
      self.linear_04 = self.linear_(100, 50)
      self.dropout = nn.Dropout(p=.2)
      self.linear_05 = nn.Linear(50, 1)

    def linear_(self, in_chan, out_chan):
        return nn.Sequential(
            nn.Linear(in_chan, out_chan),
            nn.BatchNorm1d(out_chan),
            nn.ReLU()
        )
    def forward(self, x):
      x = self.linear_01(x)
      x = self.linear_02(x)
      x = self.linear_03(x)
      x = self.linear_04(x)
      x = self.dropout(x)
      x = self.linear_05(x)
      return x

На вход будем подавать вектор, размером `300` (разме обученного эмбеддинга), 10 эпох, в качестве оптимизтора возьмем `AdamW`, лосс - `BCE`-лосс с логитами

In [ ]:
vector_size = dev.word2vec.vector_size
lr = 3e-4
num_epochs = 10
model = TextClassficator(vector_size)
model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr, weight_decay=0.001)

In [ ]:
model

TextClassficator(
  (linear_01): Sequential(
    (0): Linear(in_features=300, out_features=200, bias=True)
    (1): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (linear_02): Sequential(
    (0): Linear(in_features=200, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (linear_03): Sequential(
    (0): Linear(in_features=100, out_features=100, bias=True)
    (1): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (linear_04): Sequential(
    (0): Linear(in_features=100, out_features=50, bias=True)
    (1): BatchNorm1d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (dropout): Dropout(p=0.2, inplace=False)
  (linear_05): Linear(in_features=50, out_features=1, bias=True)
)

Обучим модель и после - проверим производительность на тестовой выборке
После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. 

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    display(pd.Series(log).to_frame().T.style.hide_index())
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.506323,74.900000


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.497941,75.590000


Epoch 10. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.498371,75.670000


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))
print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}


Результаты не впечатляющие, но и не плохие - данные были сильно перемешаны, получили некоторый резльутат. Попробуем другие подходы к подготовке данных

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


## Embeddings for unknown words

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. 

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


### Mean of context

Для каждого незнакомого слова будем запоминать его контекст. Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. Переработаем наш `TwitterDataset` и изменим метод `get_embeddings_`: будем проверять есть ли слово в словаре, если оно отсутствует - будем проверять слова слева и справа, если они отсутствуют также - будем выдавать им значение в виде нулевого вектора. Итоговым эмбеддингом центрального слова - будет сумма его соседних слов - слева и справа. Есть вероятность получить и в таком случае нулевой вектор в виде эмбеддинга, но мы повышаем возможности получить больше информации из текста

In [ ]:
class TwitterDatasetContext(TwitterDataset):
    def __init__(self, *args):
      super().__init__(*args)
      
    def get_embeddings_(self, tokens):
        all_embd = []
        checker = [-1, len(tokens)]
        null_vector = np.zeros((1, self.word2vec.vector_size))
        for word in tokens:
          if word in self.word2vec:
            all_embd.append(self.get_embd_(word))
            # если слова нет в word2vec - запоминаем его контекст
            # если слово первое или последнее - откидываем лишние индексы
          else:
            index = tokens.index(word)
            words_idx = [index - 1, index + 1]
            words_idx = list(filter(lambda words_idx: words_idx not in checker, words_idx))
            if len(words_idx) > 1:
                prev, next = tokens[words_idx[0]], tokens[words_idx[1]]
                prev = self.get_embd_(prev) if prev in self.word2vec else null_vector
                next = self.get_embd_(next) if next in self.word2vec else null_vector
                embd = (next + prev).reshape(self.word2vec.vector_size)
                all_embd.append(embd)
            elif len(words_idx) == 1:
                embd_word = tokens[words_idx[0]]
                embd = self.get_embd_(embd_word) if embd_word in self.word2vec else null_vector
                if embd.shape != null_vector.shape:
                  all_embd.append(embd)

        embeddings = all_embd
        if len(embeddings) == 0:
          embeddings = null_vector
        else:
          embeddings = np.array(all_embd)
          if len(embeddings.shape) == 1:
            embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def get_embd_(self, word):
       return ((self.word2vec.get_vector(word) - self.mean) / self.std)

    def __len__(self):
        return self.data.shape[0]

Аналогично, загрузим наш датасет, проверим его длину и возьмем образцы для `TSNE`

In [ ]:
dev = TwitterDatasetContext(dev_data, "text", "emotion", word2vec)

In [ ]:
print(dev.__len__())

1280000


In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
tsne = TSNE(n_components=2, init='random',
            learning_rate='auto')
examples["transformed_features"] = tsne.fit_transform(np.asarray(examples['features']))

Посмотрим, как поменялось отображение наших эмюеддингов

In [ ]:
draw_vectors(examples["transformed_features"], examples["targets"])

Они все еще также плохоразделимы. Для эксперимента, возьмем туже самую архитектуру нейронной сети, применим аналогичные параметры и посмотрим, как изменится производительность нашей модели. 

In [ ]:
train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

Проведем обучения с аналогичными параметрами и посмотрим, позволит ли данный подход улучшить результаты для нашего сета

In [ ]:
vector_size = dev.word2vec.vector_size
lr = 3e-4
num_epochs = 10
model = TextClassficator(vector_size)
model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr, weight_decay=0.001)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    display(pd.Series(log).to_frame().T.style.hide_index())
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.512441,74.390000


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.503602,75.280000


Epoch 10. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.504369,75.230000


На тесте мы получили следующие результаты в первом случае, когда работали только со словами в словаре `word2vec`:

```python
{'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}
```

In [ ]:
test_loader = DataLoader(
    TwitterDatasetContext(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))
print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5000921914100647, 'Test Acc, %': 75.39}


```python

1. word2vec: {'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}
2. mean of context: {'Test Loss': 0.5000921914100647, 'Test Acc, %': 75.39}

```

Результаты примерно похожие, даже в силу случайнсти при обучении - близкие. В целом данный подход, по крайней мере, не ухудшает наши результаты

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


### TF-IDF

Попробуем два варианта с `TF-IDF`

Для начала подготовим корпус к обучению `TFIDF`. Строить будем только на основании обучающей выборки, тест включать не будем, чтобы не допускать ликов

In [ ]:
def text_preprocessing(text, tokenizer):
    tokenized = tokenizer.tokenize(text)
    joined = ' '.join(tokenized)
    resubed = re.sub(r"[^+a-z]", ' ', joined)
    final = ' '.join(resubed.split())
    return final

In [ ]:
df = dev_data['text'].progress_apply(text_preprocessing, tokenizer=tokenizer)

  0%|          | 0/1280000 [00:00<?, ?it/s]

#### TF-IDF with word2vec

Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf.


Мы получили предобработанный корпус `df`. Обучим `TF_IDF`, затем понизим разменость с помощью `SVD` разложения и соберем словарь из эмбеддинга `TF_IDF` и слова

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords)
tf_idf_values = tfidf.fit_transform(df)

In [ ]:
svd = TruncatedSVD(n_components=word2vec.vector_size, n_iter=3,
                   random_state=42)
svd.fit(tf_idf_values)
transformed = svd.transform(tf_idf_values)

In [ ]:
tfidf_dict = (dict(zip(tfidf.get_feature_names(), transformed)
                       )
)

Посмотрим пример для слова `dog`

In [ ]:
tfidf_dict['dog'][:5]

array([ 0.01876099, -0.00193544, -0.00129727, -0.00142253, -0.01062714])

Эмбеддинг получился как задумано, размерность 300, можем складывать с вектором из word2vec. Переделаем наш класс для работы такими эмбеддингами

In [ ]:
class TwitterDatasetTFIDF_emb(TwitterDataset):

    def __init__(self, *args, vectorizer):
      super().__init__(*args)
      self.vectorizer = vectorizer
      self.mean_tfidf = np.mean(list(tfidf_dict.values()), axis=0)
      self.std_tfidf = np.std(list(tfidf_dict.values()), axis=0)

    def get_embeddings_(self, tokens):
        embeddings = []
        weighted = 0
        for w in tokens:
          # если слово есть и в word2vec, и в tfidf-словаре
          # находим эмбеддинг wrod2vec и прибавляем tfidf эмбеддинг
          if w in self.word2vec and w in self.vectorizer and len(w) > 3:
            embd = (self.word2vec.get_vector(w) - self.mean) / self.std
            # берем idf из словаря, считаем tdf и находим tf/idf слова
            embd_idf = (self.vectorizer[w] - self.mean_tfidf) / self.std_tfidf
            embedding = embd + embd_idf
            embeddings.append(embedding)
          elif w in self.vectorizer and w not in self.word2vec and len(w) > 3:
            embd_idf = (self.vectorizer[w] - self.mean_tfidf) / self.std_tfidf
            embedding = embd_idf
            embeddings.append(embedding)

        embeddings = np.array(embeddings)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDatasetTFIDF_emb(dev_data, "text", "emotion", word2vec, vectorizer=tfidf_dict)

In [ ]:
print(dev.__len__())

1280000


In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
tsne = TSNE(n_components=2, init='random',
            learning_rate='auto')
examples["transformed_features"] = tsne.fit_transform(np.asarray(examples['features']))

Посмотрим, как поменялось отображение наших эмбеддингов

In [ ]:
draw_vectors(examples["transformed_features"], examples["targets"])

Данные все также неразделимы, снова будем применять нашу архитектуру нейронной сети

In [ ]:
batch_size = 1024
num_workers = 2

train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
vector_size = dev.word2vec.vector_size
lr = 3e-4
num_epochs = 10
model = TextClassficator(vector_size)
model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr, weight_decay=0.001)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    display(pd.Series(log).to_frame().T.style.hide_index())
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.524750,73.350000


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.518298,74.230000


Epoch 10. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.520127,74.160000


На тесте мы получили следующие результаты в первом случае, когда работали только со словами в словаре `word2vec`:

```python
1. word2vec: {'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}
2. mean of context: {'Test Loss': 0.5000921914100647, 'Test Acc, %': 75.39}

```

In [ ]:
test_loader = DataLoader(
    TwitterDatasetTFIDF_emb(test_data, "text", "emotion", word2vec, vectorizer=tfidf_dict), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))
print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5116476935386658, 'Test Acc, %': 74.39}


Сильных улучшений, также не наблюдаем

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


#### TF-IDF with weighted word2vec



Для каждого слова текста получим его эмбеддинг из `Tfidf` с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Применим подход взвешивания с помощью `TF-IDF эмбеддингов` для наших слова. Умножаем word2vec на `tfidf` значение и делим на общий `tfidf` для данного предложения

Мы получили предобработанный корпус `df`, теперь мы можем обучить TF_IDF и затем получить словарь, где ключом будет слово, а значением - его tfidf в корпусе

In [ ]:
tfidf = TfidfVectorizer(stop_words=stopwords)
tf_idf_values = tfidf.fit_transform(df)

In [ ]:
tfidf_dict = (dict(zip(tfidf.get_feature_names(), tfidf.idf_)
                       )
)

In [ ]:
dog = tfidf_dict['dog']
print(f'IDF для слова "dog": {dog:.2f}')

IDF для слова "dog": 6.86


Поправим наш датасет, изменим подход к рассчету эмбдедингов

In [ ]:
class TwitterDatasetTFIDF(TwitterDataset):

    def __init__(self, *args, vectorizer):
      super().__init__(*args)
      self.vectorizer = vectorizer

    def get_embeddings_(self, tokens):
        embeddings = []
        weighted = 0
        for w in tokens:
          # если слово есть и в word2vec, и в tfidf-словаре
          # находим эмбеддинг wrod2vec, умножаем на tfidf слова
          # после получения вектора предложения: делим на сумму idf для слов
          # его составляющих
          if w in self.word2vec and w in self.vectorizer and len(w) > 3:
            embd = (self.word2vec.get_vector(w) - self.mean) / self.std
            # берем idf из словаря, считаем tdf и находим tf/idf
            embd_idf = self.vectorizer[w] * (tokens.count(w) / len(tokens))
            embeddings.append(embd * embd_idf)
            weighted += embd_idf
        embeddings = np.array(embeddings)
        embeddings /= (weighted + 1e-10)
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDatasetTFIDF(dev_data, "text", "emotion", word2vec, vectorizer=tfidf_dict)

In [ ]:
print(dev.__len__())

1280000


In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
tsne = TSNE(n_components=2, init='random',
            learning_rate='auto')
examples["transformed_features"] = tsne.fit_transform(np.asarray(examples['features']))

Посмотрим, как поменялось отображение наших эмбеддингов

In [ ]:
draw_vectors(examples["transformed_features"], examples["targets"])

Данные все также неразделимы, снова будем применять нашу архитектуру нейронной сети. Появились некоторые выбивающиеся точки

In [ ]:
batch_size = 1024
num_workers = 2

train_size = math.ceil(len(dev) * 0.8)
train, valid = random_split(dev, [train_size, len(dev) - train_size])
train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, 
                          shuffle=False, drop_last=False, collate_fn=average_emb)

In [ ]:
vector_size = dev.word2vec.vector_size
lr = 3e-4
num_epochs = 10
model = TextClassficator(vector_size)
model = model.cuda()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr, weight_decay=0.001)

In [ ]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    display(pd.Series(log).to_frame().T.style.hide_index())
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.533536,72.760000


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.516650,74.130000


Epoch 10. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

Test Loss,"Test Acc, %"
0.516989,74.070000


На тесте мы получили следующие результаты в первом случае, когда работали только со словами в словаре `word2vec`:

```python
1. word2vec: {'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}
2. mean of context: {'Test Loss': 0.5000921914100647, 'Test Acc, %': 75.39}
3. TF_IDF with word2vec {'Test Loss': 0.5116476935386658, 'Test Acc, %': 74.39}
```

In [ ]:
test_loader = DataLoader(
    TwitterDatasetTFIDF(test_data, "text", "emotion", word2vec, vectorizer=tfidf_dict), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))
print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.516947696685791, 'Test Acc, %': 74.07}


Результаты получились незначительно хуже. Возможно, эт освязано со случайностью при обучении сети, но в целом, сильных потерь нет, как и улучшений. Возможно, в данном случае необходимо применять более сложные подходы, например, получение эмбеддингов из `BERT` или `Elmo`

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___


## Вывод


Мы работали с корпусом твиттов и пытались классифицировать их на негативные и позитивные с помощью различных подходов получения эмбеддингов:
```
1. word2vec предобученный
2. word2vec + context
3. word2vec + TF_IDF
3. word2vec + TF_IDF (weighted word2vec)
```


Данные были смешаны, неразделимы с помощью линейных моделей. Для решения задачи мы применяли несложную многослойную полносвязную архитектуру линейной сети. В результате обучения и тестирования различных подходов формирования эмбеддингов, мы выяснили, что `word2vec` и `word2vec + context` - дают чуть лучше результаты, нежеле использование `tf_idf + word2vec`. Возможно, если бы мы рассматривали `tfidf` в рамках классов, то результаты могли бы быть лучше. Твиты являются короткими высказывания и оценка idf для корпуса в рамках одного слова - возможно не является существенной. При обработке, мы приводили к нижнему регистру и убирали символы - а они зачастую, несут в себе негативную окраску. Для улучшения результата, возможно изменить предобработку корпуса, оставляя символы, например - множественные восклицательные знаки или буквы верхнего регистра при высказываниях. также, в качестве меры улучшения для всех вариантов, можно рассмотреть более длительное обучение (более 10 эпох, значительно больше, например, 50-100), либо передавать также смысл слова, семантическую окраску, сейчас мы работаем с расстоянием и частотой слов


Результаты


На тесте мы получили следующие результаты:

```python
1. word2vec: {'Test Loss': 0.4954747290611267, 'Test Acc, %': 75.75}
2. mean of context: {'Test Loss': 0.5000921914100647, 'Test Acc, %': 75.39}
3. TF_IDF with word2vec {'Test Loss': 0.5116476935386658, 'Test Acc, %': 74.39}
4. TF_IDF + word2vec (weighted word2vec) {'Test Loss': 0.516947696685791, 'Test Acc, %': 74.07}
```

<div style="float:left;margin:0 10px 10px 0" markdown="1">
    <a href="#footer"><img src='https://img.shields.io/badge/К содержанию-&#x21A9-blue'></a>
</div>

___
